In [1]:
import pandas as pd
import janitor
print("Libraries loaded successfully!")

Libraries loaded successfully!


In [2]:
# functions to clean and transform

def cleanRedundancies (data, columns_to_keep):

    data = data[columns_to_keep]
    data = data.drop_duplicates(subset=columns_to_keep)

    return data

def fixIntType (data, columns):

    for c in columns:
        data[c] = pd.to_numeric(data[c], errors='coerce').astype('Int64')

    return data 

def fixMixedType (data, columns):

    for c in columns:
        data[c] = data[c].astype(str).str.replace(r'[^0-9.\-]', '', regex=True)
        data[c] = pd.to_numeric(data[c], errors='coerce')

    return data

def fixDateType (data, columns):

    for c in columns:
        data[c] = pd.to_datetime(data[c], errors='coerce')

    return data

In [3]:
# execution phase

filenames = ['encounterDiagnosis.vSn','HealthCondition.vSn']

for f in filenames:
    # setting low_memory=False implies reading the whole data before inferring types
    df = pd.read_csv('data/' + f + '.csv', low_memory=False)

    # ----------- standardized clean and transform steps for all tables
    
    # initialize column names of variables relevant to research objective
    columns_to_keep = []
    columns_int_type = []
    columns_date_type = []
    columns_mixed_type = ['DiagnosisCode_calc']

    # conditions of setting column names specific to data files that will be used
    if f == 'encounterDiagnosis.vSn':
        columns_to_keep = ['EncounterDiagnosis_ID','Patient_ID','DiagnosisText_calc','DiagnosisCode_calc','DateCreated']
        columns_int_type = ['EncounterDiagnosis_ID','Patient_ID']
        columns_date_type = ['DateCreated']
        
    elif f == 'HealthCondition.vSn':
        columns_to_keep = ['HealthCondition_ID','Patient_ID','DiagnosisText_calc','DiagnosisCode_calc','DateOfOnset','SignificantNegativeFlag','ActiveInactiveFlag','DateCreated']
        columns_int_type = ['HealthCondition_ID','Patient_ID']
        columns_date_type = ['DateOfOnset','DateCreated']

    # calling clean and transform functions
    df_clean = cleanRedundancies(df, columns_to_keep)
    df_clean = fixMixedType(df_clean, columns_mixed_type)
    df_clean = fixIntType(df_clean, columns_int_type)
    df_clean = fixDateType(df_clean, columns_date_type)

    # ----------- export cleaned files in .csv
    df_clean.to_csv("data/cleaned_" + f + ".csv", index=False)
